# Practica 4 (De nou)

## Requeriments

In [9]:
import gensim
from gensim.models import Word2Vec, word2vec
from gensim.models.word2vec import PathLineSentences
from typing import Tuple, List, Optional
import os
import re

## Codi per entrenar models word2vec

In [10]:
from datasets import load_dataset

dataset = load_dataset("projecte-aina/catalan_general_crawling")
dataset = dataset['train']

c:\Users\Usuario\.pyenv\pyenv-win\versions\3.11.0\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Usuario\.pyenv\pyenv-win\versions\3.11.0\Lib\site-packages\datasets\load.py:1486: FutureWarning: The repository for projecte-aina/catalan_general_crawling contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/projecte-aina/catalan_general_crawling
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [11]:
# Getting the first 100 MB of the dataset
size = 0
for i in range(len(dataset)):
    size += len(dataset[i]['text'])
    if size > 100000000:  # 100 MB
        print('files per 100MB:', i)
        break

dataset_100MB = dataset.select(list(range(i)))


# Getting the first 500 MB of the dataset
size = 0
for j in range(len(dataset)):
    size += len(dataset[j]['text'])  # corrected from dataset[i]['text']
    if size > 500000000:  # 500 MB
        print('files per 500MB:', j)
        break

dataset_500MB = dataset.select(list(range(j)))

# Getting the first 1 GB of the dataset
size = 0
for g in range(len(dataset)):
    size += len(dataset[g]['text'])  # corrected from dataset[i]['text']
    if size > 1000000000:  # 1 GB
        print('files per 1GB:', g)
        break

dataset_1GB = dataset.select(list(range(g)))

# Getting the whole dataset
dataset_complet = dataset

files per 100MB: 57333
files per 500MB: 213432
files per 1GB: 410702


In [12]:
datasets = {}
datasets['100MB'] = dataset_100MB
datasets['500MB'] = dataset_500MB
datasets['1GB'] = dataset_1GB  
datasets['complet'] = dataset_complet

models = {}

In [13]:
stopwords_catala = [ 
    "a", "abans", "ací", "així", "alguns", "alguna", "algunes", "algú", "alhora", 
    "als", "allò", "aquell", "aquelles", "aquell", "aquells", "baix", 
    "cada", "com", "com a", "eixa", "eixes", "eixí", 
    "eixos", "el", "ella", "elles", "ell", "ells", "en", "endavant", "enfront", 
    "ens", "entre", "he", "hem", "heu", "hi", "ho", 
    "i", "igual", "iguals", "ja", "jo", "l'", "la", "les", "li", "els", "tu", "nosaltres", "vosaltres",
    "de", "del", "dels", "d'un", "d'una", "d'uns", "d'unes", "des", "des de",
    ]

# Funció per netejar i tokenitzar el text
def preprocess_text(text):
    # Neteja el text: treu caràcters no desitjats, converteix a minúscules, etc.
    text = re.sub(r'\W+', ' ', text)  # Substitueix caràcters no alfanumèrics per espais
    text = text.lower()  # Converteix a minúscules
    words = text.split()  # Tokenitza
    # elimina stopwords
    words = [word for word in words if word not in stopwords_catala]
    return words

In [ ]:
# Carrega el fitxer de text i processa'l
def create_corpus(dataset):
    corpus = []
    for line in dataset:
        words = preprocess_text(line['text'])
        if words:  # Assegura't que la línia no està buida
            corpus.append(words)
    return corpus

corpus_dict = {}
for mida in ["100MB", "500MB", "1GB", "complet"]:
    dataset = datasets[mida]
    corpus_dict[f"corpus_{mida}"] = create_corpus(dataset=dataset)
    print(f"Corpus {mida} creat")


In [95]:
def entrenar_word2vec(corpus, mida_vector=100, finestra=5, sg=1, min_count=10, workers=4, epochs=25):
    
    model = word2vec.Word2Vec(corpus, vector_size=mida_vector, window=finestra, min_count=min_count, workers=workers, epochs=epochs, sg=sg)
    
    return model

# Entrenar models per a cada mida de dataset
for mida in ["500MB", "1GB", "complet"]: # "100MB"
    corpus = corpus_dict[f"corpus_{mida}"]
    model = entrenar_word2vec(corpus)
    models[mida] = model
    model.save(f"model_{mida}.model")


In [14]:
for model in ['100MB', '500MB', '1GB', 'complet']:
    # load model
    models[model] = gensim.models.Word2Vec.load(f"model_{model}.model")
    
models['100MB'] = gensim.models.Word2Vec.load("model_100MB.model")

In [15]:
models['500MB'].wv.most_similar("casa", topn=10)

[('caseta', 0.715048611164093),
 ('família', 0.6787399053573608),
 ('llar', 0.6702232360839844),
 ('pairal', 0.6680922508239746),
 ('acollidora', 0.6579598188400269),
 ('cabana', 0.6460942625999451),
 ('bonica', 0.6448329091072083),
 ('jardí', 0.640557587146759),
 ('habitació', 0.6359531283378601),
 ('granja', 0.6318382024765015)]

## Model de Similitud de Text Semàntic

In [16]:
# Requisites
from gensim.models import TfidfModel
from gensim.utils import simple_preprocess
from gensim.corpora import Dictionary
import numpy as np
from scipy import spatial
from typing import Tuple, List

In [17]:
from datasets import load_dataset

text_semantic = load_dataset("projecte-aina/sts-ca")

c:\Users\Usuario\.pyenv\pyenv-win\versions\3.11.0\Lib\site-packages\datasets\load.py:1486: FutureWarning: The repository for projecte-aina/sts-ca contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/projecte-aina/sts-ca
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [18]:
text_semantic['train']['sentence1'][0]

'Atorga per primer cop les mencions Encarna Sanahuja a la inclusió de la perspectiva de gènere en docència Universitària'

### Inicialització

In [19]:
input_pairs = [(e["sentence1"], e["sentence2"], e["label"], ) for e in text_semantic["train"].to_list()]
input_pairs_val = [(e["sentence1"], e["sentence2"], e["label"], ) for e in text_semantic["validation"].to_list()]
input_pairs_test = [(e["sentence1"], e["sentence2"], e["label"], ) for e in text_semantic["test"].to_list()]

In [20]:
all_input_pairs = input_pairs + input_pairs_val + input_pairs_test
# Preprocesamiento de las oraciones y creación del diccionario
sentences_1_preproc = [simple_preprocess(sentence_1) for sentence_1, _, _ in all_input_pairs]
sentences_2_preproc = [simple_preprocess(sentence_2) for _, sentence_2, _ in all_input_pairs]
sentence_pairs = list(zip(sentences_1_preproc, sentences_2_preproc))
# Versión aplanada para poder entrenar el modelo
sentences_pairs_flattened = sentences_1_preproc + sentences_2_preproc
diccionario = Dictionary(sentences_pairs_flattened)

In [21]:
all_input_pairs[0]

('Atorga per primer cop les mencions Encarna Sanahuja a la inclusió de la perspectiva de gènere en docència Universitària',
 'Creen la menció M. Encarna Sanahuja a la inclusió de la perspectiva de gènere en docència universitària',
 3.5)

In [22]:
# Cálculo de los pesos TF-IDF para las oraciones pre-procesadas
corpus = [diccionario.doc2bow(sent) for sent in sentences_pairs_flattened]
modelo_tfidf = TfidfModel(corpus)

In [23]:
def map_tf_idf(sentence_preproc: List[str], dictionary: Dictionary, tf_idf_model: TfidfModel, wv_model) -> Tuple[List[np.ndarray], List[float]]:
    bow = dictionary.doc2bow(sentence_preproc)
    tf_idf = tf_idf_model[bow]
    vectors, weights = [], []
    for word_index, weight in tf_idf:
        word = dictionary.get(word_index)
        if word in wv_model:
            vectors.append(wv_model[word])
            weights.append(weight)
    return vectors, weights

def map_pairs(
        sentence_pairs: List[Tuple[str, str, float]],
        dictionary: Dictionary = None,
        tf_idf_model: TfidfModel = None,
        wv_model: Word2Vec = None,
) -> List[Tuple[Tuple[np.ndarray, np.ndarray], float]]:
    # Mapeo de los pares de oraciones a pares de vectores
    pares_vectores = []
    for i, (sentence_1, sentence_2, similitud) in enumerate(sentence_pairs):
        sentence_1_preproc = preprocess_text(sentence_1)
        sentence_2_preproc = preprocess_text(sentence_2)
        # Si usamos TF-IDF
        if tf_idf_model is not None:
            # Cálculo del promedio ponderado por TF-IDF de los word embeddings
            vectors1, weights1 = map_tf_idf(sentence_1_preproc, dictionary=dictionary, tf_idf_model=tf_idf_model, wv_model=wv_model,)
            vectors2, weights2 = map_tf_idf(sentence_2_preproc, dictionary=dictionary, tf_idf_model=tf_idf_model, wv_model=wv_model,)
            vector1 = np.average(vectors1, weights=weights1, axis=0, )
            vector2 = np.average(vectors2, weights=weights2, axis=0, )
        else:
            # Cálculo del promedio de los word embeddings
            vectors1 = [wv_model[word] for word in sentence_1_preproc if word in wv_model]
            vectors2 = [wv_model[word] for word in sentence_2_preproc if word in wv_model]
            vector1 = np.mean(vectors1, axis=0)
            vector2 = np.mean(vectors2, axis=0)
        # Añadir a la lista
        pares_vectores.append(((vector1, vector2), similitud))
    return pares_vectores

In [24]:
def pair_list_to_x_y(pair_list: List[Tuple[Tuple[np.ndarray, np.ndarray], int]]) -> Tuple[Tuple[np.ndarray, np.ndarray], np.ndarray]:
    _x, _y = zip(*pair_list)
    _x_1, _x_2 = zip(*_x)
    return (np.array(_x_1), np.array(_x_2)), np.array(_y, dtype=np.float32, )

## Model Baseline

In [25]:
# Define the Model
import tensorflow as tf

In [26]:
def build_and_compile_model_better(embedding_size: int = 300, learning_rate: float = 1e-3) -> tf.keras.Model:
    # Capa de entrada para los pares de vectores
    input_1 = tf.keras.Input(shape=(embedding_size,))
    input_2 = tf.keras.Input(shape=(embedding_size,))

    # Hidden layer
    first_projection = tf.keras.layers.Dense(
        embedding_size,
        kernel_initializer=tf.keras.initializers.Identity(),
        bias_initializer=tf.keras.initializers.Zeros(),
    )
    projected_1 =  first_projection(input_1)
    projected_2 = first_projection(input_2)
    
    # Compute the cosine distance using a Lambda layer
    def normalized_product(x):
        x1, x2 = x
        x1_normalized = tf.keras.backend.l2_normalize(x1, axis=1)
        x2_normalized = tf.keras.backend.l2_normalize(x2, axis=1)
        return x1_normalized * x2_normalized

    output = tf.keras.layers.Lambda(normalized_product)([projected_1, projected_2])
    output = tf.keras.layers.Dropout(0.1)(output)
    output = tf.keras.layers.Dense(
        16,
        activation="relu",
    )(output)
    output = tf.keras.layers.Dropout(0.2)(output)
    output = tf.keras.layers.Dense(
        1,
        activation="sigmoid",
    )(output)
    
    output = tf.keras.layers.Lambda(lambda x: x * 5)(output)
    
    # Define output
    model = tf.keras.Model(inputs=[input_1, input_2], outputs=output)

    # Compile the model
    model.compile(loss='mean_squared_error',
                  optimizer=tf.keras.optimizers.Adam(learning_rate))
    return model

In [27]:
import tensorflow as tf

def model_2(
    input_length: int = 37,
    dictionary_size: int = 1000,
    embedding_size: int = 100,
    learning_rate: float = 0.01,
    pretrained_weights: Optional[np.ndarray] = None,
    trainable: bool = False,
    use_cosine: bool = False,
    l2_regularizer: float = 1e-4,
) -> tf.keras.Model:
    # Inputs
    input_1 = tf.keras.Input((input_length,), dtype=tf.int32)
    input_2 = tf.keras.Input((input_length,), dtype=tf.int32)

    # Embedding Layer
    if pretrained_weights is None:
        embedding_layer = tf.keras.layers.Embedding(
            dictionary_size, embedding_size, input_length=input_length, mask_zero=True, embeddings_initializer='uniform'
        )
    else:
        embedding_layer = tf.keras.layers.Embedding(
            pretrained_weights.shape[0], pretrained_weights.shape[1],  # Ajustar la forma de los embeddings pre-entrenados
            input_length=input_length,
            mask_zero=True,
            embeddings_initializer=tf.keras.initializers.Constant(pretrained_weights),
            trainable=trainable,
        )

    # Embed the inputs
    embedded_1 = embedding_layer(input_1)
    embedded_2 = embedding_layer(input_2)
    # Pass through the embedding layer
    _input_mask_1, _input_mask_2 = tf.not_equal(input_1, 0), tf.not_equal(input_2, 0)
    
    # Attention Mechanism
    attention_mlp = tf.keras.Sequential([
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(16, activation='tanh', kernel_regularizer=tf.keras.regularizers.l2(l2_regularizer)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1)
    ])
    # Apply attention to each embedding
    attention_weights_1 = attention_mlp(embedded_1)  
    attention_weights_2 = attention_mlp(embedded_2) 
    # Mask the attention weights
    attention_weights_1 = tf.exp(attention_weights_1) * tf.cast(_input_mask_1[:, :, None], tf.float32)
    attention_weights_2 = tf.exp(attention_weights_2) * tf.cast(_input_mask_2[:, :, None], tf.float32)
    # Normalize attention weights
    attention_weights_1 = attention_weights_1 / tf.reduce_sum(attention_weights_1, axis=1, keepdims=True)
    attention_weights_2 = attention_weights_2 / tf.reduce_sum(attention_weights_2, axis=1, keepdims=True)
    # Compute context vectors
    projected_1 = tf.reduce_sum(embedded_1 * attention_weights_1, axis=1) 
    projected_2 = tf.reduce_sum(embedded_2 * attention_weights_2, axis=1) 
    
    if use_cosine:
        # Compute the cosine distance using a Lambda layer
        def cosine_distance(x):
            x1, x2 = x
            x1_normalized = tf.keras.backend.l2_normalize(x1, axis=1)
            x2_normalized = tf.keras.backend.l2_normalize(x2, axis=1)
            return 2.5 * (1.0 + tf.reduce_sum(x1_normalized * x2_normalized, axis=1))
        output = tf.keras.layers.Lambda(cosine_distance)([projected_1, projected_2])
    else:
        # Compute the cosine distance using a Lambda layer
        def normalized_product(x):
            x1, x2 = x
            x1_normalized = tf.keras.backend.l2_normalize(x1, axis=1)
            x2_normalized = tf.keras.backend.l2_normalize(x2, axis=1)
            return x1_normalized * x2_normalized
    
        output = tf.keras.layers.Lambda(normalized_product)([projected_1, projected_2])
        output = tf.keras.layers.Dropout(0.1)(output)
        output = tf.keras.layers.Dense(
            16,
            activation="relu",
            kernel_regularizer=tf.keras.regularizers.l2(l2_regularizer)
        )(output)
        output = tf.keras.layers.Dropout(0.2)(output)
        output = tf.keras.layers.Dense(
            1,
            activation="sigmoid",
        )(output)
        
        output = tf.keras.layers.Lambda(lambda x: x * 5)(output)
        
    # Model Definition
    model = tf.keras.Model(inputs=(input_1, input_2), outputs=output)
    model.compile(
        loss="mean_squared_error", optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate)
    )
    return model


## Diferents models de Word Embeddings

### One Hot

In [28]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [29]:
# Calcula el número de muestras de entrenamiento y validación
num_train_samples = len(input_pairs)
num_val_samples = len(input_pairs_val)
num_test_samples = len(input_pairs_test)

# Combina las muestras de entrenamiento y validación para procesarlas juntas
all_input_pairs = input_pairs + input_pairs_val + input_pairs_test

# Preprocesamiento de todas las oraciones y construcción del vocabulario
preprocessed_sentences = [preprocess_text(pair[0]) + preprocess_text(pair[1]) for pair in all_input_pairs]
vocab = set([word for sentence in preprocessed_sentences for word in sentence])
vocab_size = len(vocab)

# Codificación one-hot de cada oración preprocesada
encoded_sentences = [one_hot(' '.join(sentence), vocab_size) for sentence in preprocessed_sentences]

# Obtener la longitud máxima de las secuencias codificadas
max_seq_length = max(len(seq) for seq in encoded_sentences)

# Rellenar las secuencias codificadas para que todas tengan la misma longitud
padded_encoded_sentences = pad_sequences(encoded_sentences, maxlen=max_seq_length)

# Dividir los pares de entrada nuevamente
train_encoded_pairs = [
    (padded_encoded_sentences[i], padded_encoded_sentences[i + num_train_samples], label)
    for i, (_, _, label) in enumerate(input_pairs)
    if i + num_train_samples < len(padded_encoded_sentences)
]

val_encoded_pairs = [
    (padded_encoded_sentences[i], padded_encoded_sentences[i + num_val_samples], label)
    for i, (_, _, label) in enumerate(input_pairs_val)
    if i + num_val_samples < len(padded_encoded_sentences)
]

test_encoded_pairs = [
    (padded_encoded_sentences[i], padded_encoded_sentences[i + num_test_samples], label)
    for i, (_, _, label) in enumerate(input_pairs_test)
    if i + num_test_samples < len(padded_encoded_sentences)
]

# Entrenar el modelo con los datos codificados en one-hot
model = model_2(dictionary_size=vocab_size, input_length=max_seq_length, use_cosine=True)

# Entrenar el modelo
history = model.fit(
    x=[np.array([pair[0] for pair in train_encoded_pairs]), np.array([pair[1] for pair in train_encoded_pairs])],
    y=np.array([pair[2] for pair in train_encoded_pairs]),
    validation_data=(
        [np.array([pair[0] for pair in val_encoded_pairs]), np.array([pair[1] for pair in val_encoded_pairs])],
        np.array([pair[2] for pair in val_encoded_pairs]),
    ),
    epochs=50,  # Ajustar según sea necesario
    batch_size=32,  # Ajustar según sea necesario
    verbose=1
)


Epoch 1/50
32/32 [==============================] - 2s 20ms/step - loss: 0.8300 - val_loss: 0.8093
Epoch 2/50
32/32 [==============================] - 0s 14ms/step - loss: 0.7048 - val_loss: 1.0448
Epoch 3/50
32/32 [==============================] - 0s 14ms/step - loss: 0.4929 - val_loss: 0.9730
Epoch 4/50
32/32 [==============================] - 0s 14ms/step - loss: 0.2454 - val_loss: 0.9781
Epoch 5/50
32/32 [==============================] - 0s 14ms/step - loss: 0.2292 - val_loss: 0.9541
Epoch 6/50
32/32 [==============================] - 0s 14ms/step - loss: 0.1744 - val_loss: 0.9781
Epoch 7/50
32/32 [==============================] - 0s 14ms/step - loss: 0.1120 - val_loss: 0.9451
Epoch 8/50
32/32 [==============================] - 0s 15ms/step - loss: 0.0841 - val_loss: 0.9730
Epoch 9/50
32/32 [==============================] - 0s 14ms/step - loss: 0.0734 - val_loss: 0.9686
Epoch 10/50
32/32 [==============================] - 0s 15ms/step - loss: 0.0593 - val_loss: 0.9683
Epoch 11/

L'entrenament del model està molt sobreajustat, ja que la loss del conjunt d'entrenament a la que arriba és de gairebé 0, mentres que la loss del conjunt de validació a la que arriba és de gairebé 1, deixant d'aquesta manera una diferència molt notòria entre el conjunt de test i el de validació.

In [30]:
from sklearn.metrics import mean_squared_error, r2_score


# Obtener las predicciones del modelo en el conjunto de prueba
predictions = model.predict(x=[np.array([pair[0] for pair in test_encoded_pairs]), np.array([pair[1] for pair in test_encoded_pairs])], verbose=1)

# Calcular el coeficiente de determinación (R^2)
r2 = r2_score(np.array([pair[2] for pair in test_encoded_pairs]), predictions)

print("Coeficiente de determinación (R^2) en el conjunto de prueba:", r2)

mse = mean_squared_error(np.array([pair[2] for pair in test_encoded_pairs]), predictions)
print("\nError cuadrático medio (MSE):", mse)


16/16 [==============================] - 0s 2ms/step
Coeficiente de determinación (R^2) en el conjunto de prueba: -0.2745535267284387

Error cuadrático medio (MSE): 0.9623000786934015


Com havíem mencionat anteriorment, el model al estar tan sobreajustat no funciona bé amb dades no vistes i per tant el R2_score dona negatiu. Indicant que el model prediu els valors pitjor que predint només amb la mitjana.

### Model Word2Vec

Realitzem el model de regressio amb els embeddings dels models Word2Vec + Mean i + Mean Ponderada.

#### Word2Vec + Mean 

In [31]:
from scipy.stats import pearsonr

def mapp(word2vec, batch_size: int = 64, num_epochs: int = 64):
    
    # Imprimir los pares de vectores y la puntuación de similitud asociada
    mapped = map_pairs(input_pairs, dictionary=diccionario, wv_model=word2vec.wv)
    
    # Imprimir los pares de vectores y la puntuación de similitud asociada
    mapped_train = map_pairs(input_pairs,  dictionary=diccionario, wv_model=word2vec.wv)
    mapped_val = map_pairs(input_pairs_val, dictionary=diccionario, wv_model=word2vec.wv)
    mapped_test = map_pairs(input_pairs_test, dictionary=diccionario, wv_model=word2vec.wv)

    # Obtener las listas de train y test
    x_train, y_train = pair_list_to_x_y(mapped_train)
    x_val, y_val = pair_list_to_x_y(mapped_val)

    # Preparar los conjuntos de datos de entrenamiento y validación
    train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    train_dataset = train_dataset.shuffle(buffer_size=len(x_train)).batch(batch_size)

    val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
    val_dataset = val_dataset.batch(batch_size)
    
    model = build_and_compile_model_better(embedding_size=word2vec.vector_size)
    
    model.fit(train_dataset, epochs=num_epochs, validation_data=val_dataset, verbose=0)
    
    from scipy.stats import pearsonr
    x_test, y_test = pair_list_to_x_y(mapped_test)
    
    # Baseline
    def compute_pearson_baseline(x_, y_):
        y_pred_baseline = []
        for v1, v2 in zip(*x_):
            d = 1.0 - spatial.distance.cosine(v1, v2)
            y_pred_baseline.append(d)
        # Calcular la correlación de Pearson entre las predicciones y los datos de prueba
        correlation, _ = pearsonr(y_pred_baseline, y_.flatten())
        return correlation
    
    # Imprimir el coeficiente de correlación de Pearson
    print(f"Correlación de Pearson (baseline-train): {compute_pearson_baseline(x_train, y_train)}")
    print(f"Correlación de Pearson (baseline-validation): {compute_pearson_baseline(x_val, y_val)}")
    print(f"Correlación de Pearson (baseline-test): {compute_pearson_baseline(x_test, y_test)}\n")
    
    def compute_pearson(x_, y_):
        # Obtener las predicciones del modelo para los datos de prueba. En este ejemplo vamos a utilizar el corpus de training.
        y_pred = model.predict(x_)
        # Calcular la correlación de Pearson entre las predicciones y los datos de prueba
        correlation, _ = pearsonr(y_pred.flatten(), y_.flatten())
        return correlation

    # Imprimir el coeficiente de correlación de Pearson
    print(f"Correlación de Pearson (train): {compute_pearson(x_train, y_train)}")
    print(f"Correlación de Pearson (validation): {compute_pearson(x_val, y_val)}")
    print(f"Correlación de Pearson (test): {compute_pearson(x_test, y_test)}\n\n")

for model in ['100MB', '500MB', '1GB', 'complet']:
    print(f"Entrenant model {model}\n")
    mapp(models[model])

Entrenant model 100MB

Correlación de Pearson (baseline-train): 0.34132889471710465
Correlación de Pearson (baseline-validation): 0.2878746083296036
Correlación de Pearson (baseline-test): 0.40315940546719736

65/65 [==============================] - 0s 951us/step
Correlación de Pearson (train): 0.9726040743395692
16/16 [==============================] - 0s 901us/step
Correlación de Pearson (validation): 0.24693219167955727
16/16 [==============================] - 0s 931us/step
Correlación de Pearson (test): 0.3190721192769009


Entrenant model 500MB

Correlación de Pearson (baseline-train): 0.452860546989999
Correlación de Pearson (baseline-validation): 0.4618448540799148
Correlación de Pearson (baseline-test): 0.5399993029883255

65/65 [==============================] - 0s 994us/step
Correlación de Pearson (train): 0.961833713815724
16/16 [==============================] - 0s 1ms/step
Correlación de Pearson (validation): 0.30232934045863735
16/16 [==============================] - 0s

Els valors de les Correlacions de Pearson calculats ens indiquen que el model també està molt sobreajustat, amb valors propers al 1 en tots els models pel conjunt de train i amb valors propers a 0.4 per tots els models en els conjunts de test. El model complet obté els millors resultats pel conjunt de test, però la diferència no és gairebé notòria.  

#### Word2Vec + Mean Ponderada

In [48]:
from scipy.stats import pearsonr

def mapp(word2vec, batch_size: int = 64, num_epochs: int = 64):
    
    # Imprimir los pares de vectores y la puntuación de similitud asociada
    mapped = map_pairs(input_pairs, tf_idf_model=modelo_tfidf, dictionary=diccionario, wv_model=word2vec.wv)
    
    # Imprimir los pares de vectores y la puntuación de similitud asociada
    mapped_train = map_pairs(input_pairs,  tf_idf_model=modelo_tfidf, dictionary=diccionario, wv_model=word2vec.wv)
    mapped_val = map_pairs(input_pairs_val, tf_idf_model=modelo_tfidf, dictionary=diccionario, wv_model=word2vec.wv)
    mapped_test = map_pairs(input_pairs_test, tf_idf_model=modelo_tfidf, dictionary=diccionario, wv_model=word2vec.wv)

    # Obtener las listas de train y test
    x_train, y_train = pair_list_to_x_y(mapped_train)
    x_val, y_val = pair_list_to_x_y(mapped_val)

    # Preparar los conjuntos de datos de entrenamiento y validación
    train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    train_dataset = train_dataset.shuffle(buffer_size=len(x_train)).batch(batch_size)

    val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
    val_dataset = val_dataset.batch(batch_size)
    
    model = build_and_compile_model_better(embedding_size=word2vec.vector_size)
    
    model.fit(train_dataset, epochs=num_epochs, validation_data=val_dataset, verbose=0)
    
    from scipy.stats import pearsonr
    x_test, y_test = pair_list_to_x_y(mapped_test)
    
    # Baseline
    def compute_pearson_baseline(x_, y_):
        y_pred_baseline = []
        for v1, v2 in zip(*x_):
            d = 1.0 - spatial.distance.cosine(v1, v2)
            y_pred_baseline.append(d)
        # Calcular la correlación de Pearson entre las predicciones y los datos de prueba
        correlation, _ = pearsonr(y_pred_baseline, y_.flatten())
        return correlation
    
    # Imprimir el coeficiente de correlación de Pearson
    print(f"Correlación de Pearson (baseline-train): {compute_pearson_baseline(x_train, y_train)}")
    print(f"Correlación de Pearson (baseline-validation): {compute_pearson_baseline(x_val, y_val)}")
    print(f"Correlación de Pearson (baseline-test): {compute_pearson_baseline(x_test, y_test)}\n")
    
    def compute_pearson(x_, y_):
        # Obtener las predicciones del modelo para los datos de prueba. En este ejemplo vamos a utilizar el corpus de training.
        y_pred = model.predict(x_)
        # Calcular la correlación de Pearson entre las predicciones y los datos de prueba
        correlation, _ = pearsonr(y_pred.flatten(), y_.flatten())
        return correlation

    # Imprimir el coeficiente de correlación de Pearson
    print(f"Correlación de Pearson (train): {compute_pearson(x_train, y_train)}")
    print(f"Correlación de Pearson (validation): {compute_pearson(x_val, y_val)}")
    print(f"Correlación de Pearson (test): {compute_pearson(x_test, y_test)}\n\n")

for model in ['100MB', '500MB', '1GB', 'complet']:
    print(f"Entrenant model {model}\n")
    mapp(models[model])

Entrenant model 100MB

Correlación de Pearson (baseline-train): 0.3648063022380385
Correlación de Pearson (baseline-validation): 0.27936252945628576
Correlación de Pearson (baseline-test): 0.3922879849557813

65/65 [==============================] - 0s 688us/step
Correlación de Pearson (train): 0.97947417844422
16/16 [==============================] - 0s 730us/step
Correlación de Pearson (validation): 0.2206613558163073
16/16 [==============================] - 0s 738us/step
Correlación de Pearson (test): 0.36146354618584164


Entrenant model 500MB

Correlación de Pearson (baseline-train): 0.46535328131056514
Correlación de Pearson (baseline-validation): 0.4927845302072769
Correlación de Pearson (baseline-test): 0.5245793205439706

65/65 [==============================] - 0s 695us/step
Correlación de Pearson (train): 0.9708647046487451
16/16 [==============================] - 0s 646us/step
Correlación de Pearson (validation): 0.2878802680116447
16/16 [==============================] - 0

Els resultats segueixen estan sobreajustats i els valors de les correlacions de Pearson són gairebé els mateixos per cadascún dels models.

#### Word2Vec + Mean amb model de Linear Regression

Hem volgut veure com es comporta l'entrenament del model word2vec amb una linear Regression, ja que el model que utilitzem ens dona molts mals resultats. Amb aquest experiment volem comprovar que no es tracta de la definició del model que usem, sinó de la dificultat de fer prediccions amb aquesta incrustació de paraules.

In [19]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Exemple de preprocessament de text
def obtenir_vectores_mean(text, model_word2vec):
    tokens = text.split()
    vectors = [model_word2vec.wv[word] for word in tokens if word in model_word2vec.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model_word2vec.vector_size)

# Exemple d'entrenament del model de regressió
def entrenar_model_regressio(texts, labels, model_word2vec):
    X = np.array([np.concatenate((obtenir_vectores_mean(text1, model_word2vec), obtenir_vectores_mean(text2, model_word2vec))) 
                  for text1, text2 in texts])
    
    X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2)
    model = LinearRegression()
    model.fit(X_train, y_train)
    prediccions = model.predict(X_test)
    mse = mean_squared_error(y_test, prediccions)
    return model, mse

# Entrenar el model de regressió utilitzant un dels models Word2Vec
texts =  [(sentence1, sentence2) for sentence1, sentence2, _ in input_pairs] # textos del dataset
labels = [label for _, _, label in input_pairs]  # similituds reals
model_word2vec = Word2Vec.load("model_100MB") # utilitzar el model Word2Vec entrenat

model_regressio, mse = entrenar_model_regressio(texts, labels, model_word2vec)
print(f"MSE del model de regressió: {mse}")


MSE del model de regressió: 0.6825629832324949


Un valor d'MSE de 0.6825629832324949 indica que l'error quadràtic mitjà entre les prediccions del model i els valors reals és relativament alt. Això suggereix que el model de regressió no està fent prediccions molt precises.

In [20]:
# Funció per predir la similitud entre dos textos
def predir_similitud(text1, text2, model_regressio, model_word2vec):
    vector1 = obtenir_vectores_mean(text1, model_word2vec)
    vector2 = obtenir_vectores_mean(text2, model_word2vec)
    vector_concat = np.concatenate((vector1, vector2)).reshape(1, -1)
    prediccio = model_regressio.predict(vector_concat)
    return prediccio[0]

# Exemple de predicció
text1 = "Aquest és un text"
text2 = "Aquest és un altre text"
similitud = predir_similitud(text1, text2, model_regressio, model_word2vec)
print(f"Similitud predita: {similitud}")


Similitud predita: 2.6517136096954346


En aquest exemple hem volgut comprovar que predia el model amb dues frases realment similars per veure quant de malament s'equivoca. Efectivament, la similitud predita hauria de ser més elevada i l'error és bastant gran, predint una valor en el mig de la puntuació màxima per una predicció de gairebé el màxim de la puntuació.

### 3. spaCy

In [ ]:
!python -m spacy download ca_core_news_md

In [ ]:
import spacy

# Cargar el modelo de spaCy
nlp = spacy.load('ca_core_news_md')

def embeddings_spacy(inputs):
    # Lista para almacenar los embeddings de los textos
    embeddings = []

    # Procesar cada par de textos
    for texto1, texto2, _ in inputs:
        # Procesar el primer texto
        doc1 = nlp(texto1)
        # Obtener los embeddings del primer texto
        embeddings_texto1 = np.mean([token.vector for token in doc1])
        
        # Procesar el segundo texto
        doc2 = nlp(texto2)
        # Obtener los embeddings del segundo texto
        embeddings_texto2 = np.mean([token.vector for token in doc2])
        
        # Agregar los embeddings de ambos textos a la lista
        embeddings.append((embeddings_texto1, embeddings_texto2))
        
    return np.array(embeddings), np.array([label for _, _, label in inputs])

#embedding_cjt = {'train': embeddings_spacy(input_pairs), 'val': embeddings_spacy(input_pairs_val), 'test': embeddings_spacy(input_pairs_test)}

In [78]:
# Obtener los embeddings para cada conjunto de datos
train_embeddings, labels_train = embeddings_spacy(input_pairs)
val_embeddings, labels_val = embeddings_spacy(input_pairs_val)
test_embeddings, labels_test = embeddings_spacy(input_pairs_test)

# Definir y entrenar tu modelo con los embeddings de spaCy
model = model_2(
    input_length=max_seq_length,
    dictionary_size=len(nlp.vocab),
    embedding_size=nlp.vocab.vectors_length,
    use_cosine=True
)

# Entrenar el modelo
history = model.fit(
    x=[
        np.array([pair[0] for pair in train_embeddings]),
        np.array([pair[1] for pair in train_embeddings])
    ],
    y=labels_train,
    validation_data=(
        [
            np.array([pair[0] for pair in val_embeddings]),
            np.array([pair[1] for pair in val_embeddings])
        ], 
        labels_val
    ),
    epochs=50,        # Número de épocas
    batch_size=32,    # Tamaño del lote
    verbose=1         # Modo de verbosidad
)

# Evaluar el rendimiento del modelo
evaluation = model.evaluate(
    x=[
        np.array([pair[0] for pair in test_embeddings]),
        np.array([pair[1] for pair in test_embeddings])
    ],
    y=labels_test  # Etiquetas de prueba
)

print("Rendimiento del modelo con embeddings de spaCy:")
print("Loss:", evaluation[0])
print("Accuracy:", evaluation[1])

Epoch 1/50


ValueError: in user code:

    File "c:\Users\Usuario\.pyenv\pyenv-win\versions\3.11.0\Lib\site-packages\keras\src\engine\training.py", line 1377, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\Usuario\.pyenv\pyenv-win\versions\3.11.0\Lib\site-packages\keras\src\engine\training.py", line 1360, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Usuario\.pyenv\pyenv-win\versions\3.11.0\Lib\site-packages\keras\src\engine\training.py", line 1349, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\Usuario\.pyenv\pyenv-win\versions\3.11.0\Lib\site-packages\keras\src\engine\training.py", line 1126, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\Usuario\.pyenv\pyenv-win\versions\3.11.0\Lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None

    ValueError: Exception e

In [48]:
import spacy

# Cargar el modelo de spaCy
nlp_md = spacy.load('ca_core_news_md')

# Procesar las oraciones con spaCy para obtener los embeddings
embeddings_md = []
for sentence in preprocessed_sentences:
    doc = nlp_md(' '.join(sentence))
    sentence_embedding = np.mean([token.vector for token in doc], axis=0)
    embeddings_md.append(sentence_embedding)

# Convertir la lista de embeddings en un array de NumPy
embeddings_array_md = np.array(embeddings_md)

# Dividir los pares de entrada nuevamente
train_embeddings_pairs = [
    (embeddings_array_md[i], embeddings_array_md[i + num_train_samples], label)
    for i, (_, _, label) in enumerate(input_pairs)
    if i + num_train_samples < len(embeddings_array_md)
]

val_embeddings_pairs = [
    (embeddings_array_md[i], embeddings_array_md[i + num_val_samples], label)
    for i, (_, _, label) in enumerate(input_pairs_val)
    if i + num_val_samples < len(embeddings_array_md)
]

# Definir y entrenar tu modelo con los embeddings de spaCy
model = model_2(
    input_length=max_seq_length,
    dictionary_size=len(nlp_md.vocab),
    embedding_size=nlp_md.vocab.vectors_length,
    use_cosine=True
)
history = model.fit(
    x=[
        np.array([pair[0] for pair in train_embeddings_pairs]),
        np.array([pair[1] for pair in train_embeddings_pairs])
    ],
    y=np.array([pair[2] for pair in train_embeddings_pairs]),
    validation_data=(
        [
            np.array([pair[0] for pair in val_embeddings_pairs]),
            np.array([pair[1] for pair in val_embeddings_pairs])
        ],
        np.array([pair[2] for pair in val_embeddings_pairs])
    ),
    epochs=50,
    batch_size=32,
    verbose=1
)

# Evaluar el rendimiento del modelo
evaluation = model.evaluate(
    x=[
        np.array([pair[0] for pair in val_embeddings_pairs]),
        np.array([pair[1] for pair in val_embeddings_pairs])
    ],
    y=np.array([pair[2] for pair in val_embeddings_pairs])
)

print("Rendimiento del modelo con embeddings de spaCy:")
print("Loss:", evaluation[0])
print("Accuracy:", evaluation[1])


Epoch 1/50


InvalidArgumentError: Graph execution error:

Detected at node model_12/embedding_8/embedding_lookup defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "c:\Users\Usuario\.pyenv\pyenv-win\versions\3.11.0\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "c:\Users\Usuario\.pyenv\pyenv-win\versions\3.11.0\Lib\site-packages\traitlets\config\application.py", line 1053, in launch_instance

  File "c:\Users\Usuario\.pyenv\pyenv-win\versions\3.11.0\Lib\site-packages\ipykernel\kernelapp.py", line 736, in start

  File "c:\Users\Usuario\.pyenv\pyenv-win\versions\3.11.0\Lib\site-packages\tornado\platform\asyncio.py", line 195, in start

  File "c:\Users\Usuario\.pyenv\pyenv-win\versions\3.11.0\Lib\asyncio\base_events.py", line 604, in run_forever

  File "c:\Users\Usuario\.pyenv\pyenv-win\versions\3.11.0\Lib\asyncio\base_events.py", line 1909, in _run_once

  File "c:\Users\Usua

In [32]:
import spacy
import numpy as np

# Cargar el modelo de spaCy
nlp_md = spacy.load('ca_core_news_md')

# Procesar las oraciones con spaCy para obtener los embeddings
embeddings_md = []
for sentence in preprocessed_sentences:
    # Procesar la oración con spaCy
    doc = nlp_md(' '.join(sentence))
    # Obtener el embedding de la oración como el promedio de los embeddings de las palabras
    sentence_embedding = np.mean([token.vector for token in doc], axis=0)
    embeddings_md.append(sentence_embedding)

# Convertir la lista de embeddings en un array de NumPy
embeddings_array_md = np.array(embeddings_md)

# Dividir los pares de entrada nuevamente
train_embeddings_pairs = [
    (embeddings_array_md[i], embeddings_array_md[i + num_train_samples], label)
    for i, (_, _, label) in enumerate(input_pairs)
    if i + num_train_samples < len(embeddings_array_md)
]

val_embeddings_pairs = [
    (embeddings_array_md[i], embeddings_array_md[i + num_val_samples], label)
    for i, (_, _, label) in enumerate(input_pairs_val)
    if i + num_val_samples < len(embeddings_array_md)
]

max_seq_length = max(len(seq) for seq in embeddings_md)

In [35]:
# Obtener el tamaño del vocabulario del modelo de spaCy
vocab_size = len(nlp_md.vocab)

print("Tamaño del diccionario:", vocab_size)


Tamaño del diccionario: 14222


In [ ]:
# Entrenar el modelo con los embeddings
model = model_2(dictionary_size=20000000, input_length=max_seq_length, use_cosine=True)  # El input_dim debe ser la dimensión del embedding
history = model.fit(
    x=[np.array([pair[0] for pair in train_embeddings_pairs]), np.array([pair[1] for pair in train_embeddings_pairs])],
    y=np.array([pair[2] for pair in train_embeddings_pairs]),
    validation_data=(
        [np.array([pair[0] for pair in val_embeddings_pairs]), np.array([pair[1] for pair in val_embeddings_pairs])],
        np.array([pair[2] for pair in val_embeddings_pairs]),
    ),
    epochs=50,  # Ajustar según sea necesario
    batch_size=32,  # Ajustar según sea necesario
    verbose=1
)


### 4.RoBERTa

In [ ]:
!python -m spacy download ca_core_news_trf

In [50]:
import spacy

# Cargar el modelo de spaCy
nlp = spacy.load('ca_core_news_trf')

# Procesar el texto
text = "Tu texto aquí"
doc = nlp(text)

# Obtener el tensor de embeddings
embeddings = doc._.trf_data.tensors[-1]

# Obtener el token CLS
cls_token = embeddings[0]

# Calcular la media de los embeddings
mean_embedding = embeddings.mean(axis=0)


ValueError: [E002] Can't find factory for 'curated_transformer' for language Catalan (ca). This usually happens when spaCy calls `nlp.create_pipe` with a custom component name that's not registered on the current language class. If you're using a custom component, make sure you've added the decorator `@Language.component` (for function components) or `@Language.factory` (for class components).

Available factories: attribute_ruler, tok2vec, merge_noun_chunks, merge_entities, merge_subtokens, token_splitter, doc_cleaner, parser, beam_parser, lemmatizer, trainable_lemmatizer, entity_linker, entity_ruler, tagger, morphologizer, ner, beam_ner, senter, sentencizer, spancat, spancat_singlelabel, span_finder, future_entity_ruler, span_ruler, textcat, textcat_multilabel, ca.lemmatizer

In [49]:
import spacy
import numpy as np

# Cargar el modelo de spaCy
nlp_trf = spacy.load('ca_core_news_trf')

# Lista para almacenar los embeddings
embeddings_trf = []

# Longitud máxima para los embeddings
max_embedding_length = 100

# Procesar las oraciones con spaCy para obtener los embeddings
for sentence in preprocessed_sentences:
    # Procesar la oración con spaCy
    doc = nlp_trf(' '.join(sentence))
    # Obtener las representaciones de la última capa oculta
    embeddings_doc = doc._.trf_data.last_hidden_layer_state
    # Convertir a numpy array
    embeddings_np = np.array(embeddings_doc.data)
    # Calcular la media de los embeddings a lo largo del eje de las palabras
    sentence_embedding = np.mean(embeddings_np, axis=1)
    # Aplicar relleno al final del array si es necesario
    if sentence_embedding.shape[0] < max_embedding_length:
        # Calcular cuánto relleno se necesita
        padding_length = max_embedding_length - sentence_embedding.shape[0]
        # Agregar ceros al final del array para rellenar
        padded_embedding = np.pad(sentence_embedding, (0, padding_length), mode='constant')
        embeddings_trf.append(padded_embedding)
    else:
        embeddings_trf.append(sentence_embedding[:max_embedding_length])

# Convertir la lista de embeddings en un array de NumPy
embeddings_array_trf = np.array(embeddings_trf)

# Dividir los pares de entrada nuevamente
train_embeddings_pairs = [
    (embeddings_array_trf[i], embeddings_array_trf[i + num_train_samples], label)
    for i, (_, _, label) in enumerate(input_pairs)
    if i + num_train_samples < len(embeddings_array_trf)
]

val_embeddings_pairs = [
    (embeddings_array_trf[i], embeddings_array_trf[i + num_val_samples], label)
    for i, (_, _, label) in enumerate(input_pairs_val)
    if i + num_val_samples < len(embeddings_array_trf)
]

max_seq_length = max(len(seq) for seq in embeddings_trf)


ValueError: [E002] Can't find factory for 'curated_transformer' for language Catalan (ca). This usually happens when spaCy calls `nlp.create_pipe` with a custom component name that's not registered on the current language class. If you're using a custom component, make sure you've added the decorator `@Language.component` (for function components) or `@Language.factory` (for class components).

Available factories: attribute_ruler, tok2vec, merge_noun_chunks, merge_entities, merge_subtokens, token_splitter, doc_cleaner, parser, beam_parser, lemmatizer, trainable_lemmatizer, entity_linker, entity_ruler, tagger, morphologizer, ner, beam_ner, senter, sentencizer, spancat, spancat_singlelabel, span_finder, future_entity_ruler, span_ruler, textcat, textcat_multilabel, ca.lemmatizer

In [ ]:
# Entrenar el modelo con los embeddings
model = model_2(dictionary_size=vocab_size, input_length=max_seq_length, use_cosine=True)  # El input_dim debe ser la dimensión del embedding
history = model.fit(
    x=[np.array([pair[0] for pair in train_embeddings_pairs]), np.array([pair[1] for pair in train_embeddings_pairs])],
    y=np.array([pair[2] for pair in train_embeddings_pairs]),
    validation_data=(
        [np.array([pair[0] for pair in val_embeddings_pairs]), np.array([pair[1] for pair in val_embeddings_pairs])],
        np.array([pair[2] for pair in val_embeddings_pairs]),
    ),
    epochs=50,  # Ajustar según sea necesario
    batch_size=32,  # Ajustar según sea necesario
    verbose=1
)


### 5. RoBERTa fine-tuned

In [80]:
from transformers import pipeline, AutoTokenizer
from scipy.special import logit

model = 'projecte-aina/roberta-base-ca-v2-cased-sts'
tokenizer = AutoTokenizer.from_pretrained(model)
pipe = pipeline('text-classification', model=model, tokenizer=tokenizer)

c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [81]:
def prepare(sentence_pairs):
    sentence_pairs_prep = []
    for s1, s2 in sentence_pairs:
        sentence_pairs_prep.append(f"{tokenizer.cls_token} {s1}{tokenizer.sep_token}{tokenizer.sep_token} {s2}{tokenizer.sep_token}")
    return sentence_pairs_prep

In [85]:
fine_runed_corpus_semantic = corpus_semantic.copy()
for i, (frase1, frase2) in enumerate(fine_runed_corpus_semantic):
    frase1 = ' '.join(frase1)
    frase2 = ' '.join(frase2)
    fine_runed_corpus_semantic[i] = (frase1, frase2)

In [92]:
from sklearn.metrics import mean_squared_error, r2_score

predicciones = pipe(prepare(fine_runed_corpus_semantic), add_special_tokens=False)

# convert back to scores to the original 0 and 5 interval
for prediction in predicciones:
    prediction['score'] = logit(prediction['score'])

scores = [pred['score'] for pred in predicciones]

mse = mean_squared_error(semantic_score, scores)
r2 = r2_score(semantic_score, scores)

print("Error cuadrático medio (MSE):", mse)
print("Coeficiente de determinación (R^2):", r2)


Error cuadrático medio (MSE): 0.19488246370881346
Coeficiente de determinación (R^2): 0.7218913654462078


In [ ]:
mse = mean_squared_error(semantic_score_test, predicciones)

# Calcular el coeficiente de determinación (R^2)
r2 = r2_score(semantic_score_test, predicciones)

print("Error cuadrático medio (MSE):", mse)
print("Coeficiente de determinación (R^2):", r2)

In [87]:
fine_runed_corpus_semantic_test = corpus_semantic_test.copy()
for i, (frase1, frase2) in enumerate(fine_runed_corpus_semantic_test):
    frase1 = ' '.join(frase1)
    frase2 = ' '.join(frase2)
    fine_runed_corpus_semantic_test[i] = (frase1, frase2)

In [91]:
from sklearn.metrics import mean_squared_error, r2_score

predicciones = pipe(prepare(fine_runed_corpus_semantic_test), add_special_tokens=False)

# convert back to scores to the original 0 and 5 interval
for prediction in predicciones:
    prediction['score'] = logit(prediction['score'])

scores = [pred['score'] for pred in predicciones]

mse = mean_squared_error(semantic_score_test, scores)
r2 = r2_score(semantic_score_test, scores)

print("Error cuadrático medio (MSE):", mse)
print("Coeficiente de determinación (R^2):", r2)

Error cuadrático medio (MSE): 0.3876424155120614
Coeficiente de determinación (R^2): 0.48657282822706216


Observem que l'error quadràtic és significativament inferior en comparació amb els altres models, fet que indica una millor ajustament del model. A més, destaca que el coeficient de determinació R2 és notablement elevat, suggerint una alta capacitat del model per explicar la variabilitat de les dades observades.

## Model amb embeddings entrenables

In [56]:
import tensorflow as tf

def model_2_wv_rnd(
    input_length: int = 37,
    dictionary_size: int = 1000,
    embedding_size: int = 100,
    learning_rate: float = 0.01,
    pretrained_weights: Optional[np.ndarray] = None,
    trainable: bool = False,
    use_cosine: bool = False,
    l2_regularizer: float = 1e-4,
) -> tf.keras.Model:
    # Inputs
    input_1 = tf.keras.Input((input_length,), dtype=tf.int32)
    input_2 = tf.keras.Input((input_length,), dtype=tf.int32)

    # Embedding Layer
    if pretrained_weights is None:
        embedding = tf.keras.layers.Embedding(
            dictionary_size, embedding_size, input_length=input_length, mask_zero=True, embeddings_initializer='uniform'
        )
    else:
        # Obtener la matriz de embeddings del modelo Word2Vec
        word_vectors = pretrained_weights.wv
        embedding_matrix = np.zeros((len(word_vectors), embedding_size))
        for i in range(len(word_vectors)):
            embedding_matrix[i] = word_vectors[word_vectors.index_to_key[i]]
            
        dictionary_size = embedding_matrix.shape[0]
        embedding_size = embedding_matrix.shape[1]
        initializer = tf.keras.initializers.Constant(embedding_matrix)
        embedding = tf.keras.layers.Embedding(
            dictionary_size,
            embedding_size,
            input_length=input_length,
            mask_zero=True,
            embeddings_initializer=initializer,
            trainable=trainable,
        )

    # Embed the inputs
    embedded_1 = embedding(input_1)
    embedded_2 = embedding(input_2)
    # Pass through the embedding layer
    _input_mask_1, _input_mask_2 = tf.not_equal(input_1, 0), tf.not_equal(input_2, 0)
    
    # Attention Mechanism
    attention_mlp = tf.keras.Sequential([
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(16, activation='tanh', kernel_regularizer=tf.keras.regularizers.l2(l2_regularizer)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1)
    ])
    # Apply attention to each embedding
    attention_weights_1 = attention_mlp(embedded_1)  
    attention_weights_2 = attention_mlp(embedded_2) 
    # Mask the attention weights
    attention_weights_1 = tf.exp(attention_weights_1) * tf.cast(_input_mask_1[:, :, None], tf.float32)
    attention_weights_2 = tf.exp(attention_weights_2) * tf.cast(_input_mask_2[:, :, None], tf.float32)
    # Normalize attention weights
    attention_weights_1 = attention_weights_1 / tf.reduce_sum(attention_weights_1, axis=1, keepdims=True)
    attention_weights_2 = attention_weights_2 / tf.reduce_sum(attention_weights_2, axis=1, keepdims=True)
    # Compute context vectors
    projected_1 = tf.reduce_sum(embedded_1 * attention_weights_1, axis=1) 
    projected_2 = tf.reduce_sum(embedded_2 * attention_weights_2, axis=1) 
    
    if use_cosine:
        # Compute the cosine distance using a Lambda layer
        def cosine_distance(x):
            x1, x2 = x
            x1_normalized = tf.keras.backend.l2_normalize(x1, axis=1)
            x2_normalized = tf.keras.backend.l2_normalize(x2, axis=1)
            return 2.5 * (1.0 + tf.reduce_sum(x1_normalized * x2_normalized, axis=1))
        output = tf.keras.layers.Lambda(cosine_distance)([projected_1, projected_2])
    else:
         # Compute the cosine distance using a Lambda layer
        def normalized_product(x):
            x1, x2 = x
            x1_normalized = tf.keras.backend.l2_normalize(x1, axis=1)
            x2_normalized = tf.keras.backend.l2_normalize(x2, axis=1)
            return x1_normalized * x2_normalized
    
        output = tf.keras.layers.Lambda(normalized_product)([projected_1, projected_2])
        output = tf.keras.layers.Dropout(0.1)(output)
        output = tf.keras.layers.Dense(
            16,
            activation="relu",
            kernel_regularizer=tf.keras.regularizers.l2(l2_regularizer)
        )(output)
        output = tf.keras.layers.Dropout(0.2)(output)
        output = tf.keras.layers.Dense(
            1,
            activation="sigmoid",
        )(output)
        
        output = tf.keras.layers.Lambda(lambda x: x * 5)(output)
        
    # Model Definition
    model = tf.keras.Model(inputs=(input_1, input_2), outputs=output)
    model.compile(
        loss="mean_squared_error", optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate)
    )
    return model

baseline_model = model_2(use_cosine = True)
baseline_model.compile()
baseline_model.summary()


Model: "model_13"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_27 (InputLayer)       [(None, 37)]                 0         []                            
                                                                                                  
 input_28 (InputLayer)       [(None, 37)]                 0         []                            
                                                                                                  
 embedding_1 (Embedding)     (None, 37, 100)              100000    ['input_27[0][0]',            
                                                                     'input_28[0][0]']            
                                                                                                  
 tf.math.not_equal_2 (TFOpL  (None, 37)                   0         ['input_27[0][0]']     

In [ ]:
from collections import defaultdict
from tensorflow.keras.preprocessing.sequence import pad_sequences


# Tokenitzar i preparar les seqüències d'entrada per als models
def tokenitzar(text):
    return text.lower().split()

def construir_diccionari(texts):
    word_to_index = defaultdict(lambda: len(word_to_index))
    for text in texts:
        tokens = tokenitzar(text)
        for token in tokens:
            word_to_index[token]
    return dict(word_to_index)

def text_to_sequence(text, word_to_index):
    tokens = tokenitzar(text)
    return [word_to_index[token] for token in tokens]

In [52]:
##### TRAIN #####

# Suposem que tens els teus texts i labels
texts = [(sentence1, sentence2) for sentence1, sentence2, _ in input_pairs]
labels = [label for _, _, label in input_pairs]

# Obtenim tots els textos del dataset
all_texts = [text for pair in input_pairs for text in pair[:2]]
word_to_index = construir_diccionari(all_texts)

# Convertir tots els textos a seqüències d'índexs
input_sequences = [(text_to_sequence(pair[0], word_to_index), text_to_sequence(pair[1], word_to_index)) for pair in input_pairs]

# max length of the sequences
max_len = max(max(len(seq[0]), len(seq[1])) for seq in input_sequences) 

# Padding de les seqüències d'índexs
input_1_sequences = pad_sequences([seq[0] for seq in input_sequences], maxlen=max_len, padding='post')
input_2_sequences = pad_sequences([seq[1] for seq in input_sequences], maxlen=max_len, padding='post')

# Convertir les seqüències i labels a arrays numpy
input_1_sequences = np.array(input_1_sequences)
input_2_sequences = np.array(input_2_sequences)
labels = np.array(labels)

# Ajustar la mida del diccionari basat en el nombre de paraules úniques
dictionary_size = len(word_to_index)

#### TEST ####
texts_test = [(sentence1, sentence2) for sentence1, sentence2, _ in input_pairs_test]
labels_test = [label for _, _, label in input_pairs_test]

# Obtenim tots els textos del dataset
all_texts_test = [text for pair in input_pairs_test for text in pair[:2]]
word_to_index_test = construir_diccionari(all_texts_test)

# Convertir tots els textos a seqüències d'índexs
input_sequences_test = [(text_to_sequence(pair[0], word_to_index_test), text_to_sequence(pair[1], word_to_index_test)) for pair in input_pairs_test]

# max length of the sequences
max_len_test = max(max(len(seq[0]), len(seq[1])) for seq in input_sequences_test) 

# Padding de les seqüències d'índexs
input_1_sequences_test = pad_sequences([seq[0] for seq in input_sequences_test], maxlen=max_len, padding='post')
input_2_sequences_test = pad_sequences([seq[1] for seq in input_sequences_test], maxlen=max_len, padding='post')

# Convertir les seqüències i labels a arrays numpy
input_1_sequences_test = np.array(input_1_sequences_test)
input_2_sequences_test= np.array(input_2_sequences_test)
labels_test = np.array(labels_test)

##### VALIDATION #####
# Suposem que tens els teus texts i labels
texts_val = [(sentence1, sentence2) for sentence1, sentence2, _ in input_pairs_val]
labels_val = [label for _, _, label in input_pairs_val]

# Obtenim tots els textos del dataset
all_texts_val = [text for pair in input_pairs_val for text in pair[:2]]
word_to_index_val = construir_diccionari(all_texts_val)

# Convertir tots els textos a seqüències d'índexs
input_sequences_val = [(text_to_sequence(pair[0], word_to_index_val), text_to_sequence(pair[1], word_to_index_val)) for pair in input_pairs_val]

# max length of the sequences
max_len_val = max(max(len(seq[0]), len(seq[1])) for seq in input_sequences_val) 

# Padding de les seqüències d'índexs
input_1_sequences_val = pad_sequences([seq[0] for seq in input_sequences_val], maxlen=max_len, padding='post')
input_2_sequences_val = pad_sequences([seq[1] for seq in input_sequences_val], maxlen=max_len, padding='post')


# Convertir les seqüències i labels a arrays numpy
input_1_sequences_val = np.array(input_1_sequences_val)
input_2_sequences_val= np.array(input_2_sequences_val)
labels_val = np.array(labels_val)

In [53]:
batch_size = 64

# Crear Dataset de TensorFlow per a les dades d'entrenament
train_dataset = tf.data.Dataset.from_tensor_slices(((input_1_sequences, input_2_sequences), labels))
train_dataset = train_dataset.shuffle(buffer_size=len(labels)).batch(batch_size)

# Crear Dataset de TensorFlow per a les dades de validació
val_dataset = tf.data.Dataset.from_tensor_slices(((input_1_sequences_val, input_2_sequences_val), labels_val))
val_dataset = val_dataset.batch(batch_size)

# Crear Dataset de TensorFlow per a les dades de test
test_dataset = tf.data.Dataset.from_tensor_slices(((input_1_sequences_test, input_2_sequences_test), labels_test))
test_dataset = test_dataset.batch(batch_size)

### Resultats

#### Word2Vec

In [57]:
word2vec_embedding_models = {}
for model in models:
    word2vec_embedding_models[f"{model}"] = model_2_wv_rnd(pretrained_weights=models[model], trainable=True, use_cosine=True)

In [59]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

def entrenament_wv(model: gensim.models.word2vec, input_1_test, input_2_test, labels_test, batch_size: int = 64):
    
    model.fit([input_1_sequences, input_2_sequences], labels, epochs=50, batch_size = batch_size, 
              validation_data=([input_1_sequences_val, input_2_sequences_val], labels_val), verbose=0)

    # Evaluate en el train
    mse_word2vec = model.evaluate(train_dataset)
    print(f"MSE del model amb Word2Vec Embeddings en train: {mse_word2vec}")

    # Prediccions del test
    prediccions_word2vec = model.predict([input_1_test, input_2_test])

    # MSE
    mse_word2vec = mean_squared_error(labels_test, prediccions_word2vec)
    
    print('\n\n')
    print(f"MSE del model amb Word2Vec Embeddings en test: {mse_word2vec}")
    print(f"R^2 del model amb Word2Vec Embeddings en test: {r2_score(labels_test, prediccions_word2vec)}\n")
    
for word2vec_model in word2vec_embedding_models:
    model = word2vec_embedding_models[word2vec_model]
    print(f"Entrenament embeddings pel model {word2vec_model}")
    datasets = entrenament_wv(model, input_1_sequences_test, input_2_sequences_test, labels_test, batch_size=64)
    
    

Entrenament embeddings pel model 100MB
33/33 [==============================] - 0s 1ms/step - loss: 0.0193
MSE del model amb Word2Vec Embeddings en train: 0.01931370235979557
16/16 [==============================] - 0s 1ms/step



MSE del model amb Word2Vec Embeddings en test: 2.2849441930538372
R^2 del model amb Word2Vec Embeddings en test: -2.0263778878504257
Entrenament embeddings pel model 500MB


KeyboardInterrupt: 

#### Random Init

In [152]:
model_random = model_2(dictionary_size=dictionary_size, use_cosine=True)

model_random.fit([input_1_sequences, input_2_sequences], labels, epochs=50, batch_size = batch_size, 
              validation_data=([input_1_sequences_val, input_2_sequences_val], labels_val), verbose=1)

# Evaluate en el train
mse_word2vec = model.evaluate(train_dataset)
print(f"MSE del model amb Word2Vec Embeddings en train: {mse_word2vec}")

# Prediccions del test
prediccions_word2vec = model.predict([input_1_sequences_test, input_1_sequences_test])

# MSE
mse_word2vec = mean_squared_error(labels_test, prediccions_word2vec)

print('\n\n')
print(f"MSE del model amb Word2Vec Embeddings en test: {mse_word2vec}")
print(f"R^2 del model amb Word2Vec Embeddings en test: {r2_score(labels_test, prediccions_word2vec)}")

Epoch 1/50
33/33 [==============================] - 1s 17ms/step - loss: 2.7145 - val_loss: 4.2570
Epoch 2/50
33/33 [==============================] - 0s 12ms/step - loss: 1.4605 - val_loss: 3.2239
Epoch 3/50
33/33 [==============================] - 0s 12ms/step - loss: 1.6264 - val_loss: 3.3220
Epoch 4/50
33/33 [==============================] - 0s 12ms/step - loss: 1.4782 - val_loss: 3.4054
Epoch 5/50
33/33 [==============================] - 0s 12ms/step - loss: 1.5328 - val_loss: 2.9246
Epoch 6/50
33/33 [==============================] - 0s 12ms/step - loss: 1.5845 - val_loss: 2.7458
Epoch 7/50
33/33 [==============================] - 0s 12ms/step - loss: 1.5066 - val_loss: 3.4012
Epoch 8/50
33/33 [==============================] - 0s 12ms/step - loss: 1.4396 - val_loss: 3.0944
Epoch 9/50
33/33 [==============================] - 0s 12ms/step - loss: 1.2986 - val_loss: 2.5471
Epoch 10/50
33/33 [==============================] - 0s 12ms/step - loss: 1.2878 - val_loss: 3.5059
Epoch 11/

## CONCLUSIONS

Finalment, observem que gairebé tots els models estan sobreajustats i prediuen amb molt mala precisió. Tanmateix, el model de RoBERTa fine tuned 


Si ordenem els models usats de millor a pitjor rendiment ens quedem amb la següent llista:
- Roberta-fine-tuned
- Robert 
- Spacy
- Word2Vec Entrenables
- 